In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
print("imported")

Mounted at /content/drive
imported


In [2]:
#/content/drive/MyDrive/Colab Notebooks/incident_event_log.csv
df_total=pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/incident_event_log.csv")
df_total['incident_state'].unique()

array(['New', 'Resolved', 'Closed', 'Active', 'Awaiting User Info',
       'Awaiting Problem', 'Awaiting Vendor', 'Awaiting Evidence', '-100'],
      dtype=object)

In [3]:
df_major=df_total.loc[df_total['incident_state']=='Active']

In [4]:
df_total=df_total.head(5000)
df_major =df_total.loc[df_total['incident_state'] =='Active']

In [5]:
df=df_total
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   number                   5000 non-null   object
 1   incident_state           5000 non-null   object
 2   active                   5000 non-null   bool  
 3   reassignment_count       5000 non-null   int64 
 4   reopen_count             5000 non-null   int64 
 5   sys_mod_count            5000 non-null   int64 
 6   made_sla                 5000 non-null   bool  
 7   caller_id                5000 non-null   object
 8   opened_by                5000 non-null   object
 9   opened_at                5000 non-null   object
 10  sys_created_by           5000 non-null   object
 11  sys_created_at           5000 non-null   object
 12  sys_updated_by           5000 non-null   object
 13  sys_updated_at           5000 non-null   object
 14  contact_type             5000 non-null  

In [6]:
#select the cols which is exactly required for finding outlier
col=['opened_at','category','subcategory']
df=df[col]
#extract only date part from Date
df['Date']=pd.to_datetime(df['opened_at']).dt.date
df['time']=pd.to_datetime(df['opened_at']).dt.time
df['opened_at']=pd.to_datetime(df['opened_at'])
df['week_number']=df['opened_at'].dt.week
df['month']=df['opened_at'].dt.month
df['year']=df['opened_at'].dt.year


In [7]:
#selecting the cols which is exactly required for finding outliers 
col=['opened_at','category','incident_state']
df_major=df_major[col]
#extact the date part from Datetime
df_major['Date']=pd.to_datetime(df_major['opened_at']).dt.date
df_major=df_major[['category','Date','incident_state']]

In [8]:
df_major.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1241 entries, 5 to 4986
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   category        1241 non-null   object
 1   Date            1241 non-null   object
 2   incident_state  1241 non-null   object
dtypes: object(3)
memory usage: 38.8+ KB


In [9]:
#Find selecting cols for daily weekly monthly and CIs
df=df[['subcategory','category','Date','week_number','month','year']]
df['cmdb_category']=df['subcategory'].astype(str) +'@@'+df['category'].astype(str)
df_major=df_major.rename({'category' : 'cmdb_ci'},axis=1)


In [10]:
df=df.dropna()
df_major=df_major.dropna()

In [11]:
df_major.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1241 entries, 5 to 4986
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cmdb_ci         1241 non-null   object
 1   Date            1241 non-null   object
 2   incident_state  1241 non-null   object
dtypes: object(3)
memory usage: 38.8+ KB


In [12]:
#function which explains the business logic for finding outlier CI in daily weekly and monthly basis
#fun receiving 3 input parameters as dataset,freq(daily,weekly,monthly) & percentile value
def outlierPredication(input_data,frequency,input_percentile,major_incident):
    df_threshold_day=pd.DataFrame(columns=['cmdb_category','threshold_day'])
    df_threshold_week=pd.DataFrame(columns=['cmdb_category','threshold_week'])
    #one empty Data Frame for storing results for Daily,weekly or monthly
    data_final=pd.DataFrame()
    #var which stores unique CI Items
    #cmdb_ci_unique= input_data['cmdb_ci'].unique()
    df_new=input_data.drop_duplicates(['category','subcategory'])
    cmdb_ci_unique=df_new['subcategory'].astype(str) + '@@'+df_new['category'].astype(str)
    cmdb_ci_unique=cmdb_ci_unique.tolist()
    #iterating through all CI Items
    for ci_item in cmdb_ci_unique:
        #checking condition  for Daily Basis
        if frequency == 'daily':
            #selecting the Dates & Item col required for Daily Basis
            data=input_data[['Date','cmdb_category']]
            #for each item it is grouping  by Dates and find counts and finally reseting 
            data=data[data['cmdb_category']==ci_item].groupby(['Date']).agg({'cmdb_category':'count'}).reset_index()
            #assigning count value to col "count"
            data['count']=data.cmdb_category
            # assigning CI Items to col cmdb_ci
            data['cmdb_category']=ci_item
            #checking condition for weekly basis
        #elif frequency == 'weekly':
            #selecting the week number,year & CI Item col required for weekly basis
            data_week= input_data[['week_number','year','cmdb_category']]
            #For each Item it is grouping by year and week number and find counts and finaly
            data_week=data_week[data_week['cmdb_category']==ci_item].groupby(['year','week_number']).agg({'cmdb_category':'count'}).reset_index()
            #assigning count value to col count
            data_week['count']=data_week.cmdb_category
            #assigning CI Items to col 'cmdb_ci'
            data_week['cmdb_category']=ci_item
            #checking condition for montly basis
        elif frequency == 'monthly' :
            #selecting the week number,year & CI Item col required for monthly basis
            data= input_data[['month','year','cmdb_category']]
            #For each Item it is grouping by year and month number and find counts and finaly
            data=data[data['cmdb_ci']==ci_item].groupby(['year','month']).agg({'cmdb_category':'count'}).reset_index()
            #assigning count value to col count
            data['count']=data.cmdb_category
            #assigning CI Items to col 'cmdb_ci'
            data['cmdb_category']=ci_item
        else:
            print("Enter the Frequncy as Daily weekly or monthly")
        #finding the percentile value of each count to decide outlier or not
        per= np.percentile(data['count'],input_percentile)
        #one empty list for storing the flag value as normal or outlier
        l=[]
        #comparing the percentile value of each count to decide outlier or not
        for each in data['count']:
            if (each>per):
                l.append('outlier')
            else :
                l.append('normal')
        #assigning flag values(normal/outlier to column) "flag"
        data['flag_day']=l
        #finding the percentlie value of each count to decide outlier or not
        per_week =np.percentile(data_week['count'],input_percentile)
        #one empty list for staring the flag value as normala or outlier
        l_week=[]
        #print(data_week.info())
        
        #comparing the percentile value with each count by iterating through loop
        for each in data_week['count']:
            if(each>per):
                l_week.append('outlier')
            else:
                l_week.append('normal')
        data_week['flag_week'] =l_week
        #appending the result data(Daily/weekly/monthly count values & item) to the empty Data Frame(data_final)
        data_final =data_final.append(data)
        #category = input_data['category']
        #empty Data frame to store Item name and corresponding threshols values
        #df_per
        df_per=pd.DataFrame([[ci_item,per]],columns=['cmdb_category','threshold_day'])
        df_per_week=pd.DataFrame([[ci_item,per_week]],columns=['cmdb_category','threshold_week'])
        
        #appending the result Items& threshold value to the emty Data Frame
        df_threshold_day =df_threshold_day.append(df_per)
        df_threshold_week = df_threshold_week.append(df_per_week)
        #print(df_threshold_week.info())
        df_threshold_week[['category','cmdb_ci']] = df_threshold_week.cmdb_category.str.split("@@",expand=True)
    data_final =pd.merge(data_final,df_threshold_day,on=['cmdb_category'],how='left')
    data_final[['category','cmdb_ci']] =data_final.cmdb_category.str.split("@@",expand=True)
    data_final =pd.merge(data_final,major_incident,on=['cmdb_ci','Date'],how='left')
    data_final['incident_major_state'] =np.where(data_final['incident_state'] == "Active",1,0)
    data_final['Date'] =data_final['Date'].astype("datetime64[ns]")
    data_final=data_final.rename({'incident_major_state':'incident_state'},axis=1)
    del data_final['cmdb_category']
    del df_threshold_week['cmdb_category']
    return data_final,df_threshold_day,df_threshold_week
    
        
        
            
        
            
            
    
    
    
    

In [13]:
data,df_threshold_day,df_threshold_week=outlierPredication(df,'daily',90,df_major)

In [14]:
data.head()

,Date,count,flag_day,threshold_day,category,cmdb_ci,incident_state,incident_state
0,2016-01-03,46,outlier,40.2,Subcategory 170,Category 55,Active,1
1,2016-01-03,46,outlier,40.2,Subcategory 170,Category 55,Active,1
2,2016-01-03,46,outlier,40.2,Subcategory 170,Category 55,Active,1
3,2016-01-03,46,outlier,40.2,Subcategory 170,Category 55,Active,1
4,2016-01-03,46,outlier,40.2,Subcategory 170,Category 55,Active,1


In [15]:
df_threshold_day.head()

,cmdb_category,threshold_day
0,Subcategory 170@@Category 55,40.2
0,Subcategory 215@@Category 40,32.2
0,Subcategory 125@@Category 20,60.8
0,Subcategory 97@@Category 9,4.0
0,Subcategory 168@@Category 53,52.8


In [16]:
df_threshold_week.head()

,threshold_week,category,cmdb_ci
0,40.2,Subcategory 170,Category 55
0,32.2,Subcategory 215,Category 40
0,60.8,Subcategory 125,Category 20
0,4.0,Subcategory 97,Category 9
0,52.8,Subcategory 168,Category 53
